In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import joblib
import os
os.chdir('Resources/')

In [2]:
df = pd.read_csv('21_(SYS)_CR_Encrypted_User_Data.csv')
df.head()

,Age,Sex,ChestPainType,RestingBloodPressure,Cholesterol,FastingBloodSugar,RestingECG,MaximumHeartRate,ExerciseAngina,Oldpeak,ST_Slope
0,(772523829744533212740799159515826760645071663...,(355177180331047040885024852757263415615519042...,(224436594894728602290402909525830464957741853...,(326457864273896520460148966107046997137438203...,(130426705480598194446708801820813261373596934...,(189591083162661194892336445309806339506986633...,(127414375695420507470733348977382700955112250...,(738446602330756912296276253670931446470492308...,(204613550346969139380607350312663471791231518...,NaN,(332699227897874388819012923104546325477140195...


In [3]:
custom_scaler = joblib.load('13_CC_Scaler.joblib')

df_scaled = custom_scaler.transform(df)

df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

print(df_scaled)

        Age       Sex  ChestPainType  RestingBloodPressure  Cholesterol  \
0 -1.097775 -1.854141       -0.64371             -1.440979     0.538046   

   FastingBloodSugar  RestingECG  MaximumHeartRate  ExerciseAngina  Oldpeak  \
0           0.639877   -0.838135         -0.896268       -1.025914      NaN   

   ST_Slope  
0  1.534482  


g:\Thesis - Paper\Resources\custom_tuple_scaler.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  c2_values = X.applymap(self._extract_c2)


In [4]:
model = joblib.load('12_SP_Model_RF_XGB.joblib')

In [5]:
rf = model['rf']
xgb = model['xgb']

rf_pred = rf.predict(df_scaled)
xgb_pred = xgb.predict(df_scaled)

print("Random Forest Predictions:", rf_pred)
print("XGBoost Predictions:", xgb_pred)

prediction = [r if r == x else r for r, x in zip(rf_pred, xgb_pred)]
print("Combined Prediction:", prediction)

Random Forest Predictions: [0]
XGBoost Predictions: [0]
Combined Prediction: [0]


In [6]:
rf_prob = rf.predict_proba(df_scaled)
xgb_prob = xgb.predict_proba(df_scaled)

prediction_prob = np.round((rf_prob + xgb_prob) / 2 * 100, 2)
prediction_prob

array([[95.86,  4.14]])

In [7]:
prediction_prob_class_1 = rf.predict_proba(df_scaled)[:, 1][0]
feature_importance = rf.feature_importances_
feature_contributions = np.round(feature_importance * prediction_prob_class_1 * 100, 2)
feature_contributions

array([0.42, 0.22, 1.02, 0.52, 0.29, 0.21, 0.35, 0.29, 0.23, 0.39, 1.11])

In [8]:
prediction_value = int(prediction[0])

output_data = [prediction_value] + [prediction_prob[0][0]] + [prediction_prob[0][1]] + feature_contributions.tolist()
output_data


[0,
 95.86,
 4.14,
 0.42,
 0.22,
 1.02,
 0.52,
 0.29,
 0.21,
 0.35,
 0.29,
 0.23,
 0.39,
 1.11]

In [9]:
pd.DataFrame([output_data]).to_csv("22_(SYS)_CC_Predicted_User_Output_Data.csv", index=False, header=False)